# #17 Rigid Constraints (links)
<i>Create rigid constraints (links) between deck beams and pier legs</i>
***

In [ ]:
# Input Params
spans_lengths = [5, 10, 5] #m
pier_height   = 4 #m
deck_offset = 0.5
loading = 10.0  #kN/m   Uniformly distributed load

#### Connect to LUSAS Modeller

In [ ]:
from shared.LPI import *
lusas = get_lusas_modeller()
if lusas.existsDatabase():
    raise Exception("This script will create a new model. Please save and close the current model and try again")


#### Create a new model

In [ ]:
# Create a new model
lusas.newProject("Structural", "Rigid Constraints.mdl")
# Get a reference to the model database
db = lusas.getDatabase()
# Set the vertical axis
db.setVerticalDir("Z")
# Set the unit system
db.setModelUnits("kN,m,t,s,C")

# Set view to isometric (top side view)
lusas.view().setIsometric()

#### Create Model Geometry

In [ ]:
''' Create deck lines'''
deck_lines : list[IFLine] = []

cur_x = 0
for length in spans_lengths:
    # Get the geometry data object
    geomData = lusas.geometryData().setAllDefaults()
    # set the options for creating points from coordinates
    geomData.setLowerOrderGeometryType("coordinates")
    # Set the coordinates of the first point
    geomData.addCoords(cur_x, 0, 0)
    # Set the coordinates of the second point
    geomData.addCoords(cur_x + length, 0, 0)
    # Create the line object from the geometry data
    objs = db.createLine(geomData) # Modeller returns an object set containing all the lines created, in this case we only have 1.
    # Get the line
    deck_lines.append(objs.getObject("Lines"))
    # Increment the current x position
    cur_x += length

print(f"Created {len(deck_lines)} deck lines")

In [ ]:
''' Create pier lines'''
pier_lines : list[IFLine] = []

cur_x = 0
for i in range(0, len(spans_lengths)-1):
    cur_x += spans_lengths[i]
    # Get the geometry data object with defaults
    geomData = lusas.geometryData().setAllDefaults()
    # set the options for creating points from coordinates
    geomData.setLowerOrderGeometryType("coordinates")
    # Set the coordinates of the first point
    geomData.addCoords(cur_x, 0, -deck_offset)
    # Set the coordinates of the second point
    geomData.addCoords(cur_x, 0, -deck_offset-pier_height)
    # Create the line object from the geometry data
    objs = db.createLine(geomData) # Modeller returns an object set containing all the lines created, in this case we only have 1.
    # Get the line
    pier_lines.append(objs.getObject("Lines"))

print(f"Created {len(pier_lines)} deck lines")

#### Create Model Attributes

In [ ]:
''' Create a mesh attribute'''
meshAttr = db.createMeshLine("Beam Mesh")
# Set the element type and number of elements (3 thick beam elements here)
meshAttr.setNumber("BMI21", 3)

In [ ]:
''' Create a geometric attribute'''
geomAttr = db.createGeometricLine("Beam Geometry")
# Set the beam section properties
geomAttr.setBeam(0.0125, 0.0004573, 0.00002347, 0.0, 0.00000121, 0.00532608, 0.00755776, 0.0, 0.0, 0)

In [ ]:
''' Create a material attribute'''
materialAttr = db.createIsotropicMaterial("Steel", 209_000_000, 0.3, 7.8, 10e-6)

In [ ]:
''' Create a support attribute '''
fixedSupport = db.createSupportStructural("Fully Fixed")
# set the freedoms F=Free, R=Restrained
fixedSupport.setStructural("R", "R", "R", "R", "R", "R", "F", "F", "F")

''' Create a support attribute '''
pinnedSupport = db.createSupportStructural("Pinned")
# set the freedoms 
pinnedSupport.setStructural("F", "R", "R", "F", "F", "F", "F", "F", "F")

In [ ]:
''' Create a beam load attribute '''
loadAttr = db.createLoadingBeamDistributed("UDL")
loadAttr.setBeamDistributed("Parametric", "Global", "beam")
loadAttr.addRow(0.0, 0.0, 0.0, -loading, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -loading, 0.0, 0.0, 0.0)

#### Assign attributes to the model geometry

In [ ]:
''' Assign the attributes to the deck_lines '''
# get the assignment object
assignment = lusas.assignment().setAllDefaults()

# Assign the mesh
meshAttr.assignTo(deck_lines, assignment)
# Update mesh to ensure the geometric attribute can later be visualised
db.updateMesh()

# Assign the geometry
geomAttr.assignTo(deck_lines, assignment)
# Assign the material
materialAttr.assignTo(deck_lines, assignment)
# Assign the loading
loadAttr.assignTo(deck_lines, assignment)

In [ ]:
''' Assign the supports to the points of the line '''
# Assign the pinned support to the first point
pinnedSupport.assignTo(deck_lines[0].getStartPoint(), assignment)
# Assign the pinned support to the last point
pinnedSupport.assignTo(deck_lines[-1].getEndPoint(), assignment)

In [ ]:
''' Assign the attributes to the pier_lines '''
# get the assignment object
assignment = lusas.assignment().setAllDefaults()

# Assign the mesh
meshAttr.assignTo(pier_lines, assignment)
# Update mesh to ensure the geometric attribute can later be visualised
db.updateMesh()

# Assign the geometry
geomAttr.assignTo(pier_lines, assignment)
# Assign the material
materialAttr.assignTo(pier_lines, assignment)

''' Assign the supports to the end point of the lines '''
for line in pier_lines:
    # Assign the fixed support to the first point
    fixedSupport.assignTo(line.getEndPoint(), assignment)

#### Create and assign the rigid links

In [ ]:
# Create rigid links between the pier top and the deck points

for i in range(0, len(pier_lines)):
    # Create a new attribute for each connection
    rigid_link_attr = db.createRigidLinkConstraint(f"Rigid Link Pier {i+1}")

    # Add in an object set
    # - the point at the top of the pier and
    # - the corresponding deck point
    objs = lusas.newObjectSet()
    objs.add(pier_lines[i].getStartPoint())
    objs.add(deck_lines[i].getEndPoint())

    # Get the default assignment object
    assignment = lusas.assignment().setAllDefaults()
    # Specify the rigid constraints to apply to all loadcases in Analysis 1
    assignment.setLoadsetAll("Analysis 1")
    assignment.setLcDependentConstraintsOn()

    # Assign the rigid links
    rigid_link_attr.assignTo(objs)


<H2>Solving the Analysis</H2>

Solve the analysis and open the generated results:

In [ ]:
# Solve analysis 1
db.getAnalysis("Analysis 1").solve(True)
# Open all results
db.openAllResults(False)

Set the diagrams to Force/Moments:

In [ ]:
# Ensure the diagrams layer is inserted
lusas.view().insertDiagramsLayer()
# Make sure the results transform is set to none
lusas.view().diagrams().setResultsTransformNone()
# Set the diagrams to Force/Moments
lusas.view().diagrams().setResults("Force/Moment - Thick 3D Beam", "My")
